# 06_Keras泰坦尼克号上的旅客数据集介绍

## 1 数据集下载

In [2]:
import urllib.request  #用于下载文件
import os              #导入os模块，用于确认文件是否存在

In [3]:
#下载数据集
url="http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls"   #设置下载网址
filepath="data/titanic3.xls"                         #设置存储文件的路径
if not os.path.isfile(filepath):                     #判断文件不存在就会下载文件     
    result=urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)

## 2 读取数据并进行数据预处理

In [4]:
import numpy
import pandas as pd      #pandas为python提供了DataFrame功能，可以很方便的读取数据

In [5]:
all_df=pd.read_excel(filepath)        #读取titanic3.xls

In [6]:
all_df[:2]        #查看前两项数据

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


survival(是否生存，0=否，1=是)，pclass(舱等，1=头等舱，2=二等舱，3=三等舱)，silbsp（手足或配偶也在船上数量），parch（双亲或子女也在船上数量），ticket（船票号码），fare（旅客费用），cabin（舱位号码），embarked（登船港口）

survival是label字段，其余为特征字段

In [7]:
#选择需要的字段到DataFrame
cols=['survived','name','pclass','sex','age','sibsp','parch','fare','embarked']
all_df=all_df[cols]

In [8]:
all_df[:2]

,survived,name,pclass,sex,age,sibsp,parch,fare,embarked
0,1,"Allen, Miss. Elisabeth Walton",1,female,29.0000,0,0,211.3375,S
1,1,"Allison, Master. Hudson Trevor",1,male,0.9167,1,2,151.5500,S


需要处理的问题：

1）name字段在预测阶段会使用，训练时不需要，必须先删除

2）age和fare字段存在null值，必须将null值改为平均值

3）sex字段是文字，必须转换为0与1

4）embarked字段有3个分类C、Q、S，必须使用One-Hot Encoding进行转换

## 3 数据预处理

In [9]:
#删除name字段
df=all_df.drop(['name'], axis=1)

In [10]:
#找出含null值的字段
all_df.isnull().sum()

survived      0
name          0
pclass        0
sex           0
age         263
sibsp         0
parch         0
fare          1
embarked      2
dtype: int64

In [11]:
#将字段为null的数据替换成平均值
age_mean=df['age'].mean()
df['age']=df['age'].fillna(age_mean)

fare_mean=df['fare'].mean()
df['fare']=df['fare'].fillna(fare_mean)

In [12]:
#转换性别字段
df['sex']=df['sex'].map({'female':0,'male':1}).astype(int)

In [13]:
#将embarked字段进行一维有效编码转换
x_OneHot_df=pd.get_dummies(data=df,columns=["embarked"])   #data表示要转换的DataFrame,columns表示要转换的字段

In [14]:
#查看转换后的DataFrame
x_OneHot_df[:2]

,survived,pclass,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,1,1,0,29.0000,0,0,211.3375,0,0,1
1,1,1,1,0.9167,1,2,151.5500,0,0,1


## 4 将DataFrame转换为Array

In [15]:
#将DataFrame转换为Array
ndarray = x_OneHot_df.values

In [16]:
ndarray.shape

(1309, 10)

In [17]:
#查看前两项数据
ndarray[:2]

array([[  1.    ,   1.    ,   0.    ,  29.    ,   0.    ,   0.    ,
        211.3375,   0.    ,   0.    ,   1.    ],
       [  1.    ,   1.    ,   1.    ,   0.9167,   1.    ,   2.    ,
        151.55  ,   0.    ,   0.    ,   1.    ]])

第0个字段是label,第1个字段及之后的字段是features

In [18]:
#提取features与label
Label=ndarray[:,0]            #':'表示提取所有项数，'0'提取第0个字段
Features=ndarray[:,1:]        #':'表示提取所有项数，'1:'提取第1至最后字段，是特征字段 

In [19]:
#查看前两项label标签字段
Label[:2]

array([1., 1.])

In [20]:
#查看前两项features特征字段
Features[:2]

array([[  1.    ,   0.    ,  29.    ,   0.    ,   0.    , 211.3375,
          0.    ,   0.    ,   1.    ],
       [  1.    ,   1.    ,   0.9167,   1.    ,   2.    , 151.55  ,
          0.    ,   0.    ,   1.    ]])

由以上结果可知，数值特征字段单位不同，数字差异很大，没有一个共同标准。标准化可以让所有数字都在0和1之间。

## 5 将特征字段标准化

In [21]:
from sklearn import preprocessing    #sklearn提供preprocessing数据预处理模块

In [22]:
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))  #设置标准化之后的范围在0与1之间

In [23]:
scaledFeatures = minmax_scale.fit_transform(Features)    #传入参数Features（特征字段）进行标准化

In [24]:
scaledFeatures[:2]    #查看标准化之后的特征字段前两项数据

array([[0.        , 0.        , 0.36116884, 0.        , 0.        ,
        0.41250333, 0.        , 0.        , 1.        ],
       [0.        , 1.        , 0.00939458, 0.125     , 0.22222222,
        0.2958059 , 0.        , 0.        , 1.        ]])

## 6 将数据分为训练数据与测试数据

In [26]:
msk = numpy.random.rand(len(all_df)) < 0.8     #按8:2的比例使用numpy.random.rand产生msk
train_df = all_df[msk]                       #产生训练数据，为全部数据的80%
test_df = all_df[~msk]                       #产生测试数据，为全部数据的20%

In [27]:
#创建PreprocessData函数进行数据的预处理
def PreprocessData(raw_df):
    #将字段为null的数据替换成平均值
    df=raw_df.drop(['name'],axis=1)
    age_mean = df['age'].mean()
    df['age'] = df['age'].fillna(age_mean)
    fare_mean = df['fare'].mean()
    df['fare'] = df['fare'].fillna(fare_mean)
    #转换性别字段
    df['sex']=df['sex'].map({'female':0,'male':1}).astype(int)
    #将embarked字段进行一维有效编码转换
    x_OneHot_df=pd.get_dummies(data=df,columns=["embarked"])   #data表示要转换的DataFrame,columns表示要转换的字段
    
    #将DataFrame转换为Array
    ndarray = x_OneHot_df.values
    #提取features与label
    Label=ndarray[:,0]            #':'表示提取所有项数，'0'提取第0个字段
    Features=ndarray[:,1:]        #':'表示提取所有项数，'1:'提取第1至最后字段，是特征字段 
    
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))  #设置标准化之后的范围在0与1之间
    scaledFeatures = minmax_scale.fit_transform(Features)    #传入参数Features（特征字段）进行标准化
    
    return scaledFeatures,Label

In [28]:
#调用函数对训练数据和测试数据进行预处理
train_Features,train_Label=PreprocessData(train_df)
test_Features,tes_Label=PreprocessData(test_df)

In [29]:
#查看预处理后训练数据的特征字段
train_Features[:2]

array([[0.        , 0.        , 0.36116884, 0.        , 0.        ,
        0.41250333, 0.        , 0.        , 1.        ],
       [0.        , 1.        , 0.00939458, 0.125     , 0.22222222,
        0.2958059 , 0.        , 0.        , 1.        ]])

In [30]:
#查看预处理后训练数据的标签字段
train_Label[:2]

array([1., 1.])